<a href="https://colab.research.google.com/github/MusabUmama/TSLA_Stock_Price_Prediction/blob/main/Tesla_Stock_Price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/tesla_historical_data_jan_to_aug_2023.csv')

df.fillna(df.mean(), inplace=True)

df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['Close/Last'] = df['Close/Last'].str.replace('$', '').str.replace(',', '')

df['Close/Last'] = pd.to_numeric(df['Close/Last'], errors='coerce')

In [23]:
df['Daily_Return'] = df['Close/Last'].pct_change()

def calculate_moving_average(data, window):
    return data.rolling(window=window, min_periods=1).mean()

df['7_Day_MA'] = calculate_moving_average(df['Close/Last'], window=7)
df['30_Day_MA'] = calculate_moving_average(df['Close/Last'], window=30)

In [24]:
print(df.head())

        Date  Close/Last     Volume     Open       High      Low  \
0 2023-09-01      245.01  132541600  $257.26  $259.0794  $242.01   
1 2023-08-31      258.08  108861700  $255.98    $261.18  $255.05   
2 2023-08-30      256.90  121988400  $254.20    $260.51  $250.59   
3 2023-08-29      257.18  134047600  $238.58    $257.48  $237.77   
4 2023-08-28      238.82  107673700  $242.58    $244.38  $235.35   

   Daily_Return  7_Day_MA  30_Day_MA  
0           NaN  245.0100   245.0100  
1      0.053345  251.5450   251.5450  
2     -0.004572  253.3300   253.3300  
3      0.001090  254.2925   254.2925  
4     -0.071390  251.1980   251.1980  
